In [1]:
import pandas as pd
import numpy as np
from instructions import load_df_from_pkl, save_df_as_pkl
from handling_outliers import removing_iqr, removing_percentiles, zscore_outlier, modified_z_score_outlier, count_outliers, mask_outliers, replace_missing_values
import os


path = os.path.split(os.getcwd())
data_directory = os.path.join(path[0], 'data\\raw')
data_directory

'c:\\Users\\Marta\\Desktop\\Studia\\CDV\\IV semestr 2022L\\Wykorzystanie Pythona w uczeniu maszynowym\\ml_project\\project\\ML_PROJECT_2022\\data\\raw'

In [2]:
y = load_df_from_pkl(data_directory, 'y')
X = load_df_from_pkl(data_directory, 'X')

In [3]:
outliers_methods_dict = {
    #"Removing 0.1 & 0.9" : removing_percentiles,
    #"IQR" : removing_iqr,
    "Z-score" : zscore_outlier,
    "Modified Z-score" : modified_z_score_outlier
}

In [4]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_copy_3 = X.copy()
X_std = std.fit_transform(X_copy_3)

In [5]:
for method_name, method in outliers_methods_dict.items():
    X_rem = method(pd.DataFrame(X_std))
    counted = count_outliers(X_rem)
    contamination = counted[0].sum() / (X_rem.shape[0] * X_rem.shape[1]) * 100
    masked = mask_outliers(pd.DataFrame(X_std), X_rem)
    replaced = replace_missing_values(masked, 5)
    print(method_name)
    print(counted)
    print(f'Dataset contamination: {round(contamination, 2)} %')
    print('_'*10)

KeyboardInterrupt: 

In [28]:
X_rem = zscore_outlier(pd.DataFrame(X_std))
counted = count_outliers(X_rem)
contamination = counted[0].sum() / (X_rem.shape[0] * X_rem.shape[1]) * 100
masked = mask_outliers(pd.DataFrame(X_std), X_rem)
replaced = replace_missing_values(masked, 5)
print(counted)
print(f'Dataset contamination: {round(contamination, 2)} %')
print('_'*10)

(0       30
1       30
2       16
3       28
4       36
        ..
3745    27
3746    30
3747    24
3748    32
3749    23
Length: 3750, dtype: int64,            0
3185  0.0047
479   0.0046
3667  0.0045
495   0.0044
620   0.0044
...      ...
2956  0.0013
32    0.0013
3473  0.0013
1874  0.0012
1578  0.0011

[3750 rows x 1 columns])
Dataset contamination: 0.27 %
__________


In [7]:
save_df_as_pkl(replaced, data_directory, 'X_preprocessed')

###Baseline

In [29]:
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score, f1_score
from sklearn.model_selection import cross_val_score


dummy_clf = DecisionTreeClassifier(random_state=42)
cross_val_score(dummy_clf, X, y, cv=5, scoring='accuracy')

array([0.94933333, 0.93333333, 0.96133333, 0.944     , 0.94266667])

In [30]:
clf = DecisionTreeClassifier(random_state=42)
cross_val_score(clf, replaced, y, cv=5, scoring='accuracy')

array([0.95866667, 0.94266667, 0.94      , 0.944     , 0.944     ])

In [ ]:

methods = ["most_frequent", "prior", "stratified", "uniform"]
scores = {}

for method in methods:
  dummy_clf = DummyClassifier(strategy=method)
  clf = dummy_clf.fit(X_train, y_train)
  y_pred = dummy_clf.predict(X_test)
  scores[method] = dummy_clf.score(X_test, y_pred)
  print(dummy_clf.score(X_test, y_pred))
  print(balanced_accuracy_score(y_test, y_pred))
  print(f1_score(y_test, y_pred))
  print('_'*20)

scores

###Preprocessed

In [16]:
X_train, X_test, y_train, y_test = train_test_split(replaced, y, shuffle=True, random_state = 42)

In [17]:
methods = ["most_frequent", "prior", "stratified", "uniform"]
scores = {}

for method in methods:
  dummy_clf = DummyClassifier(strategy=method)
  clf = dummy_clf.fit(X_train, y_train)
  y_pred = dummy_clf.predict(X_test)
  scores[method] = dummy_clf.score(X_test, y_pred)
  print(dummy_clf.score(X_test, y_pred))
  print(balanced_accuracy_score(y_test, y_pred))
  print(f1_score(y_test, y_pred))
  print('_'*20)

scores

1.0
0.5
0.9484304932735426
____________________
1.0
0.5
0.9484304932735426
____________________
0.8113006396588486
0.4849676225716929
0.8908765652951699
____________________
0.5245202558635395
0.5179617637989515
0.6566037735849056
____________________


{'most_frequent': 1.0,
 'prior': 1.0,
 'stratified': 0.8049040511727079,
 'uniform': 0.523454157782516}